# SymMNA_demo
The circuit shown below is from W. Hayt and J. Kemmerly, *Engineering Circuit Analysis, 3rd Edition*, 1978, McGraw-Hill, Figure 3-43 (b). 

The circuit has five nodes and nine branches. There are two independent voltage sources, $V_1$ and $V_2$. The value of $V_2$ is 0 volts and the current through $V_2$ is needed for $F_1$, the current controlled current source. There is one independent current source, $I_1$.  The circuit has five resistors. The student is asked to find the current through $R_2$, which is the same as the current in $V_2$. The answer provided in the book is 0.241 Amps, which agrees with the solution obtained below.

![](SymMNA_demo.png)

The netlist for the circuit was generated by LTSpice and show below:
```
* SymMNA_demo.asc
R1 5 1 12
R2 3 2 3
R3 4 3 10
R4 5 4 2
R5 0 5 5
V1 1 0 100
I1 0 3 4
F1 0 4 V2 3
V2 1 2 0
```

The following Python modules are used in this notebook.

In [1]:
from sympy import *
import numpy as np
from tabulate import tabulate
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
import SymMNA
from IPython.display import display, Markdown, Math, Latex
init_printing()

## Load the netlist
The netlist for the circuit is pasted into the code cell below. In Python a triple-quoted string includes whitespace, tabs and newlines. The newlines characters are needed to mark the end of each SPICE statement in the netlist.

In [2]:
net_list = '''
R1 5 1 12
R2 3 2 3
R3 4 3 10
R4 5 4 2
R5 0 5 5
V1 1 0 100
I1 0 3 4
F1 0 4 V2 3
V2 1 2 0
'''

## Call the symbolic modified nodal analysis function


In [3]:
report, network_df, i_unk_df, A, X, Z = SymMNA.smna(net_list)

The network equations for the circuit can be obtained from the A, X and Z values returned from the SMNA function. The A, X and Z are formulated into equations and displayed below. Markdown is an IPython function and latex is a SymPy printing function.

In [4]:
# reform X and Z into Matrix type for printing
Xp = Matrix(X)
Zp = Matrix(Z)
temp = ''
for i in range(len(X)):
    temp += '${:s}$<br>'.format(latex(Eq((A*Xp)[i:i+1][0],Zp[i])))

Markdown(temp)

$I_{V1} + I_{V2} + \frac{v_{1}}{R_{1}} - \frac{v_{5}}{R_{1}} = 0$<br>$- I_{V2} + \frac{v_{2}}{R_{2}} - \frac{v_{3}}{R_{2}} = 0$<br>$v_{3} \cdot \left(\frac{1}{R_{3}} + \frac{1}{R_{2}}\right) - \frac{v_{4}}{R_{3}} - \frac{v_{2}}{R_{2}} = I_{1}$<br>$- I_{F1} + v_{4} \cdot \left(\frac{1}{R_{4}} + \frac{1}{R_{3}}\right) - \frac{v_{5}}{R_{4}} - \frac{v_{3}}{R_{3}} = 0$<br>$v_{5} \cdot \left(\frac{1}{R_{5}} + \frac{1}{R_{4}} + \frac{1}{R_{1}}\right) - \frac{v_{4}}{R_{4}} - \frac{v_{1}}{R_{1}} = 0$<br>$v_{1} = V_{1}$<br>$v_{1} - v_{2} = V_{2}$<br>$I_{F1} - I_{V2} f_{1} = 0$<br>

### Netlist statistics

In [5]:
print(report)

Net list report
number of lines in netlist: 9
number of branches: 9
number of nodes: 5
number of unknown currents: 3
number of RLC (passive components): 5
number of resistors: 5
number of capacitors: 0
number of inductors: 0
number of independent voltage sources: 2
number of independent current sources: 1
number of Op Amps: 0
number of E - VCVS: 0
number of G - VCCS: 0
number of F - CCCS: 1
number of H - CCVS: 0
number of K - Coupled inductors: 0



### Connectivity Matrix

In [6]:
A

⎡1                               -1                  ⎤
⎢──    0      0        0         ───       1   1   0 ⎥
⎢R₁                               R₁                 ⎥
⎢                                                    ⎥
⎢     1      -1                                      ⎥
⎢ 0   ──     ───       0          0        0  -1   0 ⎥
⎢     R₂      R₂                                     ⎥
⎢                                                    ⎥
⎢     -1   1    1     -1                             ⎥
⎢ 0   ───  ── + ──    ───         0        0   0   0 ⎥
⎢      R₂  R₃   R₂     R₃                            ⎥
⎢                                                    ⎥
⎢            -1     1    1       -1                  ⎥
⎢ 0    0     ───    ── + ──      ───       0   0   -1⎥
⎢             R₃    R₄   R₃       R₄                 ⎥
⎢                                                    ⎥
⎢-1                   -1     1    1    1             ⎥
⎢───   0      0       ───    ── + ── + ──  0   0   0 ⎥
⎢ R₁      

### Unknown voltages and currents

In [7]:
X

### Known voltages and currents

In [8]:
Z

### Network dataframe

In [9]:
network_df

,element,p node,n node,cp node,cn node,Vout,value,Vname,Lname1,Lname2
0,V1,1,0,NaN,NaN,NaN,100.0,NaN,NaN,NaN
1,V2,1,2,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,R1,5,1,NaN,NaN,NaN,12.0,NaN,NaN,NaN
3,R2,3,2,NaN,NaN,NaN,3.0,NaN,NaN,NaN
4,R3,4,3,NaN,NaN,NaN,10.0,NaN,NaN,NaN
5,R4,5,4,NaN,NaN,NaN,2.0,NaN,NaN,NaN
6,R5,0,5,NaN,NaN,NaN,5.0,NaN,NaN,NaN
7,I1,0,3,NaN,NaN,NaN,4.0,NaN,NaN,NaN
8,F1,0,4,NaN,NaN,NaN,3.0,V2,NaN,NaN


### Unknown current dataframe

In [10]:
i_unk_df

,element,p node,n node
0,V1,1,0
1,V2,1,2
2,F1,0,4


### Build the network equations

In [11]:
# Put matrices into SymPy 
X = Matrix(X)
Z = Matrix(Z)

NE_sym = Eq(A*X,Z)

# reform X and Z into Matrix type for printing
Xp = Matrix(X)
Zp = Matrix(Z)
temp = ''
for i in range(len(X)):
    temp += '${:s}$<br>'.format(latex(Eq((A*Xp)[i:i+1][0],Zp[i])))

Markdown(temp)

$I_{V1} + I_{V2} + \frac{v_{1}}{R_{1}} - \frac{v_{5}}{R_{1}} = 0$<br>$- I_{V2} + \frac{v_{2}}{R_{2}} - \frac{v_{3}}{R_{2}} = 0$<br>$v_{3} \cdot \left(\frac{1}{R_{3}} + \frac{1}{R_{2}}\right) - \frac{v_{4}}{R_{3}} - \frac{v_{2}}{R_{2}} = I_{1}$<br>$- I_{F1} + v_{4} \cdot \left(\frac{1}{R_{4}} + \frac{1}{R_{3}}\right) - \frac{v_{5}}{R_{4}} - \frac{v_{3}}{R_{3}} = 0$<br>$v_{5} \cdot \left(\frac{1}{R_{5}} + \frac{1}{R_{4}} + \frac{1}{R_{1}}\right) - \frac{v_{4}}{R_{4}} - \frac{v_{1}}{R_{1}} = 0$<br>$v_{1} = V_{1}$<br>$v_{1} - v_{2} = V_{2}$<br>$I_{F1} - I_{V2} f_{1} = 0$<br>

Turn the free symbols into SymPy variables.

In [12]:
var(str(NE_sym.free_symbols).replace('{','').replace('}',''))

## Symbolic solution
The network equations can be solved symbolically. 

In [13]:
U_sym = solve(NE_sym,X)

Display the symbolic solution

In [14]:
temp = ''
for i in U_sym.keys():
    temp += '${:s} = {:s}$<br>'.format(latex(i),latex(U_sym[i]))

Markdown(temp)

$v_{1} = V_{1}$<br>$v_{2} = V_{1} - V_{2}$<br>$v_{3} = \frac{I_{1} R_{1} R_{2} R_{3} + I_{1} R_{1} R_{2} R_{4} + I_{1} R_{1} R_{2} R_{5} + I_{1} R_{2} R_{3} R_{5} + I_{1} R_{2} R_{4} R_{5} + R_{1} R_{3} V_{1} - R_{1} R_{3} V_{2} + R_{1} R_{4} V_{1} f_{1} + R_{1} R_{4} V_{1} - R_{1} R_{4} V_{2} f_{1} - R_{1} R_{4} V_{2} + R_{1} R_{5} V_{1} f_{1} + R_{1} R_{5} V_{1} - R_{1} R_{5} V_{2} f_{1} - R_{1} R_{5} V_{2} + R_{2} R_{5} V_{1} + R_{3} R_{5} V_{1} - R_{3} R_{5} V_{2} + R_{4} R_{5} V_{1} f_{1} + R_{4} R_{5} V_{1} - R_{4} R_{5} V_{2} f_{1} - R_{4} R_{5} V_{2}}{R_{1} R_{2} + R_{1} R_{3} + R_{1} R_{4} f_{1} + R_{1} R_{4} + R_{1} R_{5} f_{1} + R_{1} R_{5} + R_{2} R_{5} + R_{3} R_{5} + R_{4} R_{5} f_{1} + R_{4} R_{5}}$<br>$v_{4} = \frac{I_{1} R_{1} R_{2} R_{4} + I_{1} R_{1} R_{2} R_{5} - I_{1} R_{1} R_{3} R_{4} f_{1} - I_{1} R_{1} R_{3} R_{5} f_{1} + I_{1} R_{2} R_{4} R_{5} - I_{1} R_{3} R_{4} R_{5} f_{1} + R_{1} R_{4} V_{1} f_{1} + R_{1} R_{4} V_{1} - R_{1} R_{4} V_{2} f_{1} - R_{1} R_{4} V_{2} + R_{1} R_{5} V_{1} f_{1} + R_{1} R_{5} V_{1} - R_{1} R_{5} V_{2} f_{1} - R_{1} R_{5} V_{2} + R_{2} R_{5} V_{1} + R_{3} R_{5} V_{1} + R_{4} R_{5} V_{1} f_{1} + R_{4} R_{5} V_{1} - R_{4} R_{5} V_{2} f_{1} - R_{4} R_{5} V_{2}}{R_{1} R_{2} + R_{1} R_{3} + R_{1} R_{4} f_{1} + R_{1} R_{4} + R_{1} R_{5} f_{1} + R_{1} R_{5} + R_{2} R_{5} + R_{3} R_{5} + R_{4} R_{5} f_{1} + R_{4} R_{5}}$<br>$v_{5} = \frac{I_{1} R_{1} R_{2} R_{5} - I_{1} R_{1} R_{3} R_{5} f_{1} + R_{1} R_{5} V_{1} f_{1} + R_{1} R_{5} V_{1} - R_{1} R_{5} V_{2} f_{1} - R_{1} R_{5} V_{2} + R_{2} R_{5} V_{1} + R_{3} R_{5} V_{1} + R_{4} R_{5} V_{1} f_{1} + R_{4} R_{5} V_{1}}{R_{1} R_{2} + R_{1} R_{3} + R_{1} R_{4} f_{1} + R_{1} R_{4} + R_{1} R_{5} f_{1} + R_{1} R_{5} + R_{2} R_{5} + R_{3} R_{5} + R_{4} R_{5} f_{1} + R_{4} R_{5}}$<br>$I_{V1} = \frac{I_{1} R_{1} R_{3} + I_{1} R_{1} R_{4} + I_{1} R_{1} R_{5} + I_{1} R_{2} R_{5} - I_{1} R_{3} R_{5} f_{1} + I_{1} R_{3} R_{5} + I_{1} R_{4} R_{5} - R_{1} V_{1} + R_{1} V_{2} - R_{2} V_{1} - R_{3} V_{1} - R_{4} V_{1} f_{1} - R_{4} V_{1} - R_{5} V_{2} f_{1}}{R_{1} R_{2} + R_{1} R_{3} + R_{1} R_{4} f_{1} + R_{1} R_{4} + R_{1} R_{5} f_{1} + R_{1} R_{5} + R_{2} R_{5} + R_{3} R_{5} + R_{4} R_{5} f_{1} + R_{4} R_{5}}$<br>$I_{V2} = \frac{- I_{1} R_{1} R_{3} - I_{1} R_{1} R_{4} - I_{1} R_{1} R_{5} - I_{1} R_{3} R_{5} - I_{1} R_{4} R_{5} + R_{1} V_{1} - R_{1} V_{2} - R_{5} V_{2}}{R_{1} R_{2} + R_{1} R_{3} + R_{1} R_{4} f_{1} + R_{1} R_{4} + R_{1} R_{5} f_{1} + R_{1} R_{5} + R_{2} R_{5} + R_{3} R_{5} + R_{4} R_{5} f_{1} + R_{4} R_{5}}$<br>$I_{F1} = \frac{- I_{1} R_{1} R_{3} f_{1} - I_{1} R_{1} R_{4} f_{1} - I_{1} R_{1} R_{5} f_{1} - I_{1} R_{3} R_{5} f_{1} - I_{1} R_{4} R_{5} f_{1} + R_{1} V_{1} f_{1} - R_{1} V_{2} f_{1} - R_{5} V_{2} f_{1}}{R_{1} R_{2} + R_{1} R_{3} + R_{1} R_{4} f_{1} + R_{1} R_{4} + R_{1} R_{5} f_{1} + R_{1} R_{5} + R_{2} R_{5} + R_{3} R_{5} + R_{4} R_{5} f_{1} + R_{4} R_{5}}$<br>

## Construct a dictionary of element values

In [15]:
element_values = SymMNA.get_part_values(network_df)

# display the component values
for k,v in element_values.items():
    print('{:s} = {:s}'.format(str(k), str(v)))

V1 = 100.0
V2 = 0.0
R1 = 12.0
R2 = 3.0
R3 = 10.0
R4 = 2.0
R5 = 5.0
I1 = 4.0
f1 = 3.0


## Numerical solution
Substitute numerical values in place of the symbolic reference designators. 

In [16]:
NE = NE_sym.subs(element_values)

Display the equations with numeric values.

In [17]:
temp = ''
for i in range(shape(NE.lhs)[0]):
    temp += '${:s} = {:s}$<br>'.format(latex(NE.rhs[i]),latex(NE.lhs[i]))

Markdown(temp)

$0 = I_{V1} + I_{V2} + 0.0833333333333333 v_{1} - 0.0833333333333333 v_{5}$<br>$0 = - I_{V2} + 0.333333333333333 v_{2} - 0.333333333333333 v_{3}$<br>$4.0 = - 0.333333333333333 v_{2} + 0.433333333333333 v_{3} - 0.1 v_{4}$<br>$0 = - I_{F1} - 0.1 v_{3} + 0.6 v_{4} - 0.5 v_{5}$<br>$0 = - 0.0833333333333333 v_{1} - 0.5 v_{4} + 0.783333333333333 v_{5}$<br>$100.0 = v_{1}$<br>$0 = v_{1} - v_{2}$<br>$0 = I_{F1} - 3.0 I_{V2}$<br>

Solve for voltages and currents.

In [18]:
U = solve(NE,X)

Display the numerical solution. Six significant digits are displayed so that results can be compared to LTSpice. 

In [19]:
table_header = ['unknown', 'mag']
table_row = []

for name, value in U.items():
    table_row.append([str(name),float(value)])

print(tabulate(table_row, headers=table_header,colalign = ('left','decimal'),tablefmt="simple",floatfmt=('5s','.6f')))

unknown           mag
---------  ----------
v1         100.000000
v2         100.000000
v3          99.276382
v4          56.864322
v5          46.934673
I_V1        -4.663317
I_V2         0.241206
I_F1         0.723618
